In [1]:
import cv2
import numpy as np
import os
from IPython.display import Video, display
import matplotlib.pyplot as plt

In [2]:
input_img  = r"C:\PERSONAL\Bala\Bala_MTECH\3 rd sem\Internship\SelfDriving\Input Images\solidWhiteCurve.jpg"
output_img = r"C:\PERSONAL\Bala\Bala_MTECH\3 rd sem\Internship\SelfDriving\Basic Perception\Output.jpg"

In [3]:
input_video  = r"C:\PERSONAL\Bala\Bala_MTECH\3 rd sem\Internship\SelfDriving\Input Video\solidWhiteRight.mp4"
output_video = r"C:\PERSONAL\Bala\Bala_MTECH\3 rd sem\Internship\SelfDriving\Basic Perception\Output.mp4"

In [4]:
#Edge Detection
def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150)
    return edges

In [5]:
#ROI
def region_of_interest(image):
    h, w = image.shape[:2]
    polygon = np.array([[(0,h), (w,h), (w//2, int(h*0.55))]])
    
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygon, 255)
    return cv2.bitwise_and(image, mask)

In [6]:
#Line Detection + Averaging
def make_coordinates(image, line):
    slope, intercept = line
    h = image.shape[0]
    y1 = h
    y2 = int(h*0.6)
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return np.array([x1,y1,x2,y2])

In [7]:
#Extract and Average Hough lines
def average_slope_intercept(image, lines):
    if lines is None:
        return []

    left = []
    right = []

    for line in lines:
        x1,y1,x2,y2 = line.reshape(4)
        slope, intercept = np.polyfit((x1,x2),(y1,y2),1)

        if slope < 0:
            left.append((slope,intercept))
        else:
            right.append((slope,intercept))

    final = []

    if len(left):
        left_avg = np.average(left, axis=0)
        final.append(make_coordinates(image, left_avg))
    if len(right):
        right_avg = np.average(right, axis=0)
        final.append(make_coordinates(image, right_avg))

    return final


In [8]:
#Lines Display
def display_lines(image, lines):
    line_img = np.zeros_like(image)

    if lines is None or len(lines)==0:
        return line_img

    for line in lines:
        if line is None: continue
        x1,y1,x2,y2 = line
        cv2.line(line_img,(x1,y1),(x2,y2),(0,255,0),8)

    return line_img

In [9]:
#lane Detection Pipeline
def pipeline(image):
    edges = canny(image)
    roi = region_of_interest(edges)

    lines = cv2.HoughLinesP(roi, 2, np.pi/180, 20, np.array([]), minLineLength=20, maxLineGap=50)

    averaged_lines = average_slope_intercept(image, lines)
    line_img = display_lines(image, averaged_lines)
    combo = cv2.addWeighted(image, 0.8, line_img, 1, 1)
    return combo

In [11]:
import os
print("PB exists:", os.path.exists(r"C:\PERSONAL\Bala\Bala_MTECH\3 rd sem\Internship\SelfDriving\Basic Perception\Models\frozen_inference_graph.pb"))
print("PBTXT exists:", os.path.exists(r"C:\PERSONAL\Bala\Bala_MTECH\3 rd sem\Internship\SelfDriving\Basic Perception\Models\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"))
print("COCO.NAMES exists:", os.path.exists(r"C:\PERSONAL\Bala\Bala_MTECH\3 rd sem\Internship\SelfDriving\Basic Perception\Models\coco.names"))

PB exists: True
PBTXT exists: True
COCO.NAMES exists: True


In [12]:
#Load COCO labels
import cv2

model = r"C:\PERSONAL\Bala\Bala_MTECH\3 rd sem\Internship\SelfDriving\Basic Perception\Models\frozen_inference_graph.pb"
config = r"C:\PERSONAL\Bala\Bala_MTECH\3 rd sem\Internship\SelfDriving\Basic Perception\Models\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"

net = cv2.dnn_DetectionModel(model, config)
net.setInputSize(320, 320)
net.setInputScale(1.0/127.5)
net.setInputMean((127.5,127.5,127.5))
net.setInputSwapRB(True)

with open(r"C:\PERSONAL\Bala\Bala_MTECH\3 rd sem\Internship\SelfDriving\Basic Perception\Models\coco.names") as f:
    classNames = f.read().split("\n")

In [13]:
print(len(classNames), classNames[:10])

81 ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light']


In [14]:
img = cv2.imread(r"C:\PERSONAL\Bala\Bala_MTECH\3 rd sem\Internship\SelfDriving\Input Images\solidWhiteCurve.jpg")

classIds, confs, boxes = net.detect(img, confThreshold=0.3)
print("classIds:", classIds)
print("confs:", confs)
print("boxes:", boxes)

classIds: [ 3  3  3  3  3  3  3  8  8  3  3  3  3  3  1  3  8 10  1 21 21  3  3  3
  3  3  3  3  8  8 21]
confs: [0.7326956  0.7057671  0.60839117 0.5900403  0.47754136 0.4477706
 0.4436142  0.43373936 0.42951724 0.42778143 0.41681582 0.41553956
 0.40002948 0.39970598 0.38649723 0.38631833 0.37126562 0.37102234
 0.36325836 0.36025107 0.35616183 0.34825498 0.32615298 0.3247697
 0.32331854 0.32227013 0.32138276 0.31230164 0.30374753 0.30189335
 0.30006737]
boxes: [[184 306  85  46]
 [103 307  70  46]
 [283 305  36  28]
 [ 56 305  28  16]
 [365 305  22  16]
 [390 306  24  17]
 [260 311  38  36]
 [184 306  85  46]
 [283 305  36  28]
 [105 307  41  19]
 [197 303  61  33]
 [576 278  21  18]
 [285 316  27  21]
 [265 306  32  24]
 [390 306  24  17]
 [424 302  23  18]
 [103 307  70  46]
 [294 283  15  14]
 [424 302  23  18]
 [424 302  23  18]
 [390 306  24  17]
 [631 279  18  14]
 [552 286  19  14]
 [ 61 310  27  17]
 [491 298  24  15]
 [147 304  35  23]
 [569 279  22  18]
 [ 34 312  24  11]
 [

In [15]:
#Image Detection
def detect_objects(image, conf=0.6):
    classIds, confs, boxes = net.detect(image, confThreshold=conf)
    objs = []

    if len(classIds) != 0:
        for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), boxes):
            
            label = classNames[classId-1]

            # ✅ Filter only vehicles
            if label not in ["car", "truck", "bus"]:
                continue

            objs.append((label, float(confidence), box))

    return objs

In [16]:
#Single Image Drawing Boxes
def draw_objects(image, objects):
    for label, confidence, box in objects:
        x, y, w, h = map(int, box)

        cv2.rectangle(image, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(image,
                    f"{label} {confidence:.2f}",
                    (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0,255,0), 2)

    return image

In [17]:
#Run on Single Image
image = cv2.imread(input_img)

objects = detect_objects(image)      # detect
result  = draw_objects(image, objects)   # annotate

cv2.imwrite(output_img, result)
print(" Saved output to:", output_img)

 Saved output to: C:\PERSONAL\Bala\Bala_MTECH\3 rd sem\Internship\SelfDriving\Basic Perception\Output.jpg


In [18]:
frame = cv2.imread(r"C:\PERSONAL\Bala\Bala_MTECH\3 rd sem\Internship\SelfDriving\Input Images\solidWhiteCurve.jpg")  
objects = detect_objects(frame)
print(objects)

[('car', 0.7326955795288086, array([184, 306,  85,  46])), ('car', 0.7057670950889587, array([103, 307,  70,  46])), ('car', 0.6083911657333374, array([283, 305,  36,  28]))]


In [19]:
#Video Object Detection Function
def detect_objects(image, conf=0.6):
    classIds, confs, boxes = net.detect(image, confThreshold=conf)
    objs = []

    if len(classIds) != 0:
        for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), boxes):
            label = classNames[classId - 1]
            if label not in ["car", "truck", "bus"]:
                continue
            objs.append((label, float(confidence), box))
    return objs

In [20]:
#Video Drawing Objects
def draw_objects(image, objects):
    for label, confidence, box in objects:
        x, y, w, h = map(int, box)
        cv2.rectangle(image, (x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(image, f"{label} {confidence:.2f}",
                    (x,y-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,(0,255,0),2)
    return image

In [21]:
#Video Perception 
def perception_pipeline(frame):
    lane_img = pipeline(frame)
    objects = detect_objects(frame)
    final = annotate_objects(lane_img, objects)
    return final

In [22]:
#Run and Saving Video 
cap = cv2.VideoCapture(input_video)

w = int(cap.get(3))
h = int(cap.get(4))
fps = cap.get(5) if cap.get(5) > 0 else 30

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video, fourcc, fps, (w,h))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    objs = detect_objects(frame)
    frame = draw_objects(frame, objs)   

    out.write(frame)

cap.release()
out.release()
print("Video saved:", output_video)

Video saved: C:\PERSONAL\Bala\Bala_MTECH\3 rd sem\Internship\SelfDriving\Basic Perception\Output.mp4
